In [ ]:
import rtsvg
import uuid
import copy
rt = rtsvg.RACETrack()

In [ ]:
#
# sankeyGeometry()
# - fm     = (x, y, h)
# - tos    = [(x, y, h), ...]
# - colors = {fm_to_tuple: hex-color, ...}
# - colors = None (default, black)
# - colors = <hex-color-as-a-string>, e.g., '#ff0000'
#
def sankeyGeometry(fm, tos, colors=None, opacity=0.1, render_exit=True, render_enters=True):
    # Sanity check
    _sum_ = fm[2]
    for to in tos: _sum_ -= to[2]
    if _sum_ != 0: raise ValueError("sankeyGeometry(): fm and tos do not sum to 100%")
    # Don't mess up the original... order it by y value
    tos = copy.deepcopy(tos)
    tos = sorted(tos, key=lambda x: x[1])
    # Draw the exit and entrances ... exit in green, entrances in red
    svg = ['<svg>']
    if render_exit:   svg.append(f'<line x1="{fm[0]}" y1="{fm[1]}" x2="{fm[0]}" y2="{fm[1]+fm[2]}" stroke="#006d00" stroke-width="4.0" />')
    if render_enters: 
        for to in tos: svg.append(f'<line x1="{to[0]}" y1="{to[1]}" x2="{to[0]}" y2="{to[1]+to[2]}" stroke="#ff0000" stroke-width="4.0" />')
    # Do the defs
    id_lu = {}
    if type(colors) is dict and fm in colors:
        svg.append('<defs>')
        for to in tos:
            if to in colors:
                id_lu[(fm, to)] = f'gradient-' + uuid.uuid4().hex
                svg.append(f'<linearGradient id="{id_lu[(fm, to)]}" x1="0%" y1="0%" x2="100%" y2="0%">')
                svg.append(f'<stop offset="0%"   stop-color="{colors[fm]}" />')
                svg.append(f'<stop offset="100%" stop-color="{colors[to]}" />')
                svg.append('</linearGradient>')
        svg.append('</defs>')
    # Most basic version of the geometry ... suffers from a narrowing in the middle which is not desired
    y = fm[1]
    for to in tos:
        _d_         = (to[0] - fm[0]) / 3.0 # push out for the bezier curve
        _fattening_ = 1.2                   # helps to fatten up the line if the geometry is too narrow
        d = f'M {fm[0]} {y} C {fm[0]+_d_} {y} {to[0]-_fattening_*_d_} {to[1]} {to[0]} {to[1]} L {to[0]} {to[1]+to[2]} C {to[0]-_d_} {to[1]+to[2]} {fm[0]+_fattening_*_d_} {y+to[2]} {fm[0]} {y+to[2]} Z'
        if   colors is None:       _color_ = '#000000'
        elif type(colors) is str:  _color_ = colors
        elif type(colors) is dict and fm in colors and to in colors: _color_ = 'url(#'+id_lu[(fm, to)]+')'
        elif type(colors) is dict and fm in colors:                  _color_ = colors[fm]
        else:                                                        _color_ = '#000000'
        svg.append(f'<path d="{d}" fill="{_color_}" fill-opacity="{opacity}" stroke="none" stroke-width="0.5" />')
        y += to[2]
    return ''.join(svg)+'</svg>'

# Example Data
_fm_  = (10, 10, 100)
_tos_ = [(100, 10, 10), (200, 50, 10), (120, 160, 80)]
svgs = [f'<svg x="0" y="0" width="512" height="256"><rect x="0" y="0" width="512" height="256" fill="#ffffff"/>']
_fm_  = (10, 10, 100)
_tos_ = [(100, 10, 10), (200, 50, 10), (120, 160, 80)]
svgs.append(sankeyGeometry(_fm_, _tos_))
_fm_  = (120, 160, 80)
_tos_ = [(200, 10, 10), (210, 80, 10), (200, 160, 60)]
svgs.append(sankeyGeometry(_fm_, _tos_, colors='#ff0000'))
_fm_       = (200,  160, 60)
_tos_      = [(300, 100, 30),(320, 150, 30)]
_color_lu_ = {(200,  160, 60):'#ff0000',
              (300,  100, 30):'#00ff00',
              (320,  150, 30):'#0000ff'}
svgs.append(sankeyGeometry(_fm_, _tos_, colors=_color_lu_, opacity=0.5))
_fm_       = (300, 100, 30)
_tos_      = [(450, 85, 15),(450, 120, 15)]
_color_lu_ = {(300,  100, 30):'#00ff00'}
svgs.append(sankeyGeometry(_fm_, _tos_, colors=_color_lu_, opacity=0.5))
_fm_       = (320,  150, 30)
_tos_      = [(450,140,15),(450, 180, 15)]
_color_lu_ = {(320,  150, 30):'#ff0000',
              (450,  180, 15):'#0000ff'}
svgs.append(sankeyGeometry(_fm_, _tos_, colors=_color_lu_, opacity=0.75))
svgs.append('</svg>')
rt.tile([''.join(svgs)])